In [ ]:
# ['California', 'Iowa', 'Kansas', 'New York', 'Texas', 'Wisconsin']


# States with just places named after themselves:
# ['Colorado', 'Florida', 'Idaho', 'Nebraska', 'Oklahoma', 'Oregon', 'Virginia']


# States with just places named after other states:
# ['Illinois', 'Indiana', 'Kentucky', 'Michigan', 'Minnesota', 'Missouri', 'New Jersey', 'North Carolina', 'Ohio', 'Pennsylvania', 'Utah']

In [1]:
import time
import geopandas
import pandas
import os
import numpy as np
from matplotlib import pyplot as plt
shapefile_basedir = '/home/idies/workspace/Temporary/raddick/census_scratch/shapefiles/2020/'
# state_name = 'Colorado'
# theplace = 'Florence'
#state_num = 8
g = 0
print('ok')

ok


In [2]:
s = time.time()
# print('getting roads...')
# roads_gdf = geopandas.GeoDataFrame()
# roads_file_list = [shapefile_basedir+'ROADS/'+x for x in os.listdir(shapefile_basedir+'ROADS/') if (x[-4:] == '.shp') and ('tl_2020_{0:02d}'.format(state_num) in x)]

# for i in range(0, len(roads_file_list)):
#     if ((np.mod(i,10) == 0) | (i == len(roads_file_list)-1)):
#         print('\tReading road file {0:,.0f} of {1:,.0f}...'.format(i+1, len(roads_file_list)))
#     roads_gdf_i = geopandas.read_file(roads_file_list[i])    
#     roads_gdf = pandas.concat((roads_gdf, roads_gdf_i), axis=0, sort=False)
# roads_gdf = roads_gdf.set_index('LINEARID')
# roads_gdf.loc[:, 'FULLNAME'] = roads_gdf['FULLNAME'].apply(lambda x: str(x))


print('getting places...')
place_gdf = geopandas.GeoDataFrame()

place_file_list = [shapefile_basedir+'PLACE/'+x for x in os.listdir(shapefile_basedir+'PLACE/') if (x[-4:] == '.shp')]# and ('tl_2020_{0:02d}'.format(state_num) in x)]

for i in range(0, len(place_file_list)):
    if (np.mod(i, 5) == 0):
        print('\treading place file {0:,.0f} of {1:,.0f}...'.format(i, len(place_file_list)))
    place_gdf_i = geopandas.read_file(place_file_list[i])
    place_gdf = pandas.concat((place_gdf, place_gdf_i), axis=0)
place_gdf.loc[:, 'INTPTLON'] = pandas.to_numeric(place_gdf['INTPTLON'])
place_gdf.loc[:, 'INTPTLAT'] = pandas.to_numeric(place_gdf['INTPTLAT'])
place_gdf.loc[:, 'STATEFP'] = pandas.to_numeric(place_gdf['STATEFP'], errors='coerce')

place_gdf.loc[:, 'GEOID'] = place_gdf['GEOID'].apply(lambda x: '1600000US'+x)
place_gdf = place_gdf.set_index('GEOID')


# print('getting water files...')
# water_gdf = geopandas.GeoDataFrame()

# # for i in range(0, len(water_file_list)):
# water_file_list = [shapefile_basedir+'AREAWATER/'+x for x in os.listdir(shapefile_basedir+'AREAWATER/') if (x[-4:] == '.shp') and ('tl_2020_{0:02d}'.format(state_num) in x)]

# for i in range(0, len(water_file_list)):
#     if (np.mod(i, 10) == 0):
#         print('\treading water file {0:,.0f} of {1:,.0f}...'.format(i, len(water_file_list)))
#     water_gdf_i = geopandas.read_file(water_file_list[i])
#     #water_gdf_i = water_gdf_i[water_gdf_i['AWATER'] >= water_area_tol]
#     #water_gdf_i = water_gdf_i.assign(STUSAB = state_codes_df[state_codes_df['STATE'] == int(waterfiles[i][89:91])]['STUSAB'].values[0])
#     water_gdf = pandas.concat((water_gdf, water_gdf_i), axis=0)
# #        print(water_gdf.head(1).T)
# #        break
# water_gdf.loc[:, 'INTPTLON'] = pandas.to_numeric(water_gdf['INTPTLON'], errors='coerce')
# water_gdf.loc[:, 'INTPTLAT'] = pandas.to_numeric(water_gdf['INTPTLAT'], errors='coerce')
# water_gdf = water_gdf.set_index('HYDROID')

print('getting states...')
state_gdf = geopandas.read_file(shapefile_basedir+'tl_2020_us_state.shp')
state_gdf.loc[:, 'INTPTLON'] = pandas.to_numeric(state_gdf['INTPTLON'])
state_gdf.loc[:, 'INTPTLAT'] = pandas.to_numeric(state_gdf['INTPTLAT'])
state_gdf.loc[:, 'STATEFP'] = pandas.to_numeric(state_gdf['STATEFP'], errors='coerce')
state_gdf = state_gdf[state_gdf['STATEFP'] < 60]

state_gdf = state_gdf.set_index('GEOID')

# # # print('\n')
# #len(roads_file_list)
e = time.time()
g += (e-s)
print('Got the following in {0:.0f} minutes {1:.1f} seconds:'.format(np.floor((e-s)/60), (e-s)%60))

#print('\tRoads: {0:,.0f}'.format(len(roads_gdf)))
print('\tPlaces: {0:,.0f}'.format(len(place_gdf)))
print('\tStates: {0:,.0f}'.format(len(state_gdf)))
#print('\tWater areas: {0:,.0f}'.format(len(water_gdf)))
print('DONE')


#place_gdf.groupby('STATEFP').size().sort_index()


getting places...
	reading place file 0 of 51...
	reading place file 5 of 51...
	reading place file 10 of 51...
	reading place file 15 of 51...
	reading place file 20 of 51...
	reading place file 25 of 51...
	reading place file 30 of 51...
	reading place file 35 of 51...
	reading place file 40 of 51...
	reading place file 45 of 51...
	reading place file 50 of 51...
getting states...
Got the following in 0 minutes 26.9 seconds:
	Places: 31,617
	States: 51
DONE


In [14]:
place_with_state_name_gdf = place_gdf.reset_index().merge(state_gdf[['STATEFP', 'NAME']].rename(columns={'NAME': 'state_name'}), how='left', on='STATEFP').set_index('GEOID')
#place_with_state_name_gdf.apply(lambda row: '{0:}, {1:}'.format(row['NAME'], row['state_name']), axis=1)

print('Finding states with place names that include the name of the state...')
place_with_state_name_gdf = place_with_state_name_gdf.assign(POP100 = np.nan)

place_with_state_name_gdf.loc['1600000US0501990', 'POP100'] = 376
place_with_state_name_gdf.loc['1600000US0609780', 'POP100'] = 14973
place_with_state_name_gdf.loc['1600000US0816000', 'POP100'] = 478961
place_with_state_name_gdf.loc['1600000US1019730', 'POP100'] = 1885
place_with_state_name_gdf.loc['1600000US1222975', 'POP100'] = 13085
place_with_state_name_gdf.loc['1600000US1639700', 'POP100'] = 64818
place_with_state_name_gdf.loc['1600000US1639610', 'POP100'] = 466
place_with_state_name_gdf.loc['1600000US1938640', 'POP100'] = 5106
place_with_state_name_gdf.loc['1600000US1938595', 'POP100'] = 74828
place_with_state_name_gdf.loc['1600000US2036000', 'POP100'] = 156607
place_with_state_name_gdf.loc['1600000US2743144', 'POP100'] = 202
place_with_state_name_gdf.loc['1600000US2743198', 'POP100'] = 661
place_with_state_name_gdf.loc['1600000US2948980', 'POP100'] = 217
place_with_state_name_gdf.loc['1600000US3133705', 'POP100'] = 7222
place_with_state_name_gdf.loc['1600000US3651000', 'POP100'] = 8804190
place_with_state_name_gdf.loc['1600000US3651011', 'POP100'] = 3244
place_with_state_name_gdf.loc['1600000US3958002', 'POP100'] = 651
place_with_state_name_gdf.loc['1600000US4055000', 'POP100'] = 681054
place_with_state_name_gdf.loc['1600000US4155200', 'POP100'] = 37572
place_with_state_name_gdf.loc['1600000US4773460', 'POP100'] = 1332
place_with_state_name_gdf.loc['1600000US4872392', 'POP100'] = 51898
place_with_state_name_gdf.loc['1600000US5182000', 'POP100'] = 459470
place_with_state_name_gdf.loc['1600000US5588150', 'POP100'] = 2942
place_with_state_name_gdf.loc['1600000US5588200', 'POP100'] = 18887


print(
    place_with_state_name_gdf[
        (place_with_state_name_gdf.apply(lambda row: row['state_name'] in row['NAME'], axis=1))
        & (
            (place_with_state_name_gdf['CLASSFP'].apply(lambda x: x in ['C1','C2','C3','C4','C5','C6','C7']))
    #        (place_with_state_name_gdf['NAME'].apply(lambda x: 'University' not in x))
    #        & (place_with_state_name_gdf['NAME'].apply(lambda x: 'College' not in x))
        )
        & (place_with_state_name_gdf['POP100'] >= 5000)
    ].apply(lambda row: '{0:}, {1:} (pop = {2:,.0f})'.format(row['NAME'], row['state_name'], row['POP100']), axis=1)
)

states_with_places_named_after_themselves_list = place_with_state_name_gdf[
        (place_with_state_name_gdf.apply(lambda row: row['state_name'] in row['NAME'], axis=1))
        & (
            (place_with_state_name_gdf['CLASSFP'].apply(lambda x: x in ['C1','C2','C3','C4','C5','C6','C7']))
    #        (place_with_state_name_gdf['NAME'].apply(lambda x: 'University' not in x))
    #        & (place_with_state_name_gdf['NAME'].apply(lambda x: 'College' not in x))
        )
        & (place_with_state_name_gdf['POP100'] >= 5000)
    ]['state_name'].drop_duplicates().tolist()

#print('\n')
print('Finding states with place names that include the name of other states...')
state_names_list = place_with_state_name_gdf['state_name'].drop_duplicates().sort_values().tolist()

other_state_names_gdf = geopandas.GeoDataFrame()
#print('Checking for places named after other states in...')
for this_state in state_names_list:
    print('\t{0:}...'.format(this_state))
    other_state_names_gdf = pandas.concat((other_state_names_gdf, place_with_state_name_gdf[
        (place_with_state_name_gdf.apply(lambda row:this_state in row['NAME'], axis=1))
        & (place_with_state_name_gdf.apply(lambda row: row['state_name'] not in row['NAME'], axis=1))
        & (
            (place_with_state_name_gdf['CLASSFP'].apply(lambda x: x in ['C1','C2','C3','C4','C5','C6','C7']))
        )
    ]), axis=0)
    
other_state_names_gdf = other_state_names_gdf.drop('1600000US0129560', axis=0)    # Georgiana, Alabama
other_state_names_gdf = other_state_names_gdf.drop('1600000US1150000', axis=0)    # Washington, DC
other_state_names_gdf = other_state_names_gdf.drop('1600000US3136015', axis=0)    # Ohiowa, NE
other_state_names_gdf = other_state_names_gdf.drop('1600000US4256408', axis=0)    # Ohiopyle, PA


place_with_state_name_gdf.loc['1600000US0139280', 'POP100'] = 180
place_with_state_name_gdf.loc['1600000US0414870', 'POP100'] = 2478
place_with_state_name_gdf.loc['1600000US0573370', 'POP100'] = 94
place_with_state_name_gdf.loc['1600000US0632506', 'POP100'] = 14149
place_with_state_name_gdf.loc['1600000US0650874', 'POP100'] = 3152
place_with_state_name_gdf.loc['1600000US0838370', 'POP100'] = 1782
place_with_state_name_gdf.loc['1600000US1081350', 'POP100'] = 1680
place_with_state_name_gdf.loc['1600000US1274575', 'POP100'] = 2364
place_with_state_name_gdf.loc['1600000US1380704', 'POP100'] = 3754
place_with_state_name_gdf.loc['1600000US1738986', 'POP100'] = 670
place_with_state_name_gdf.loc['1600000US1755379', 'POP100'] = 465
place_with_state_name_gdf.loc['1600000US1756484', 'POP100'] = 3604
place_with_state_name_gdf.loc['1600000US1774665', 'POP100'] = 101
place_with_state_name_gdf.loc['1600000US1777642', 'POP100'] = 570
place_with_state_name_gdf.loc['1600000US1778201', 'POP100'] = 1514
place_with_state_name_gdf.loc['1600000US1779033', 'POP100'] = 16071
place_with_state_name_gdf.loc['1600000US1779085', 'POP100'] = 2592
place_with_state_name_gdf.loc['1600000US1783687', 'POP100'] = 1300
place_with_state_name_gdf.loc['1600000US1848798', 'POP100'] = 32075
place_with_state_name_gdf.loc['1600000US1848816', 'POP100'] = 441
place_with_state_name_gdf.loc['1600000US1880504', 'POP100'] = 12017
place_with_state_name_gdf.loc['1600000US1919630', 'POP100'] = 142
place_with_state_name_gdf.loc['1600000US1952860', 'POP100'] = 2678
place_with_state_name_gdf.loc['1600000US1955695', 'POP100'] = 6925
place_with_state_name_gdf.loc['1600000US1956595', 'POP100'] = 498
place_with_state_name_gdf.loc['1600000US1957495', 'POP100'] = 112
place_with_state_name_gdf.loc['1600000US1982335', 'POP100'] = 7352
place_with_state_name_gdf.loc['1600000US1987240', 'POP100'] = 523
place_with_state_name_gdf.loc['1600000US2002300', 'POP100'] = 11974
place_with_state_name_gdf.loc['1600000US2111872', 'POP100'] = 83
place_with_state_name_gdf.loc['1600000US2154228', 'POP100'] = 18090
place_with_state_name_gdf.loc['1600000US2237445', 'POP100'] = 3436
place_with_state_name_gdf.loc['1600000US2279870', 'POP100'] = 742
place_with_state_name_gdf.loc['1600000US2481675', 'POP100'] = 505
place_with_state_name_gdf.loc['1600000US2681960', 'POP100'] = 716
place_with_state_name_gdf.loc['1600000US2688940', 'POP100'] = 76501
place_with_state_name_gdf.loc['1600000US2075825', 'POP100'] = 1071
place_with_state_name_gdf.loc['1600000US2746060', 'POP100'] = 1294
place_with_state_name_gdf.loc['1600000US2767288', 'POP100'] = 8421
place_with_state_name_gdf.loc['1600000US2772022', 'POP100'] = 8032
place_with_state_name_gdf.loc['1600000US2910468', 'POP100'] = 4498
place_with_state_name_gdf.loc['1600000US2938000', 'POP100'] = 508090
place_with_state_name_gdf.loc['1600000US2944174', 'POP100'] = 3199
place_with_state_name_gdf.loc['1600000US2946586', 'POP100'] = 28284
place_with_state_name_gdf.loc['1600000US2951644', 'POP100'] = 8212
place_with_state_name_gdf.loc['1600000US2953102', 'POP100'] = 4467
place_with_state_name_gdf.loc['1600000US2954848', 'POP100'] = 837
place_with_state_name_gdf.loc['1600000US2977416', 'POP100'] = 14500
place_with_state_name_gdf.loc['1600000US3077125', 'POP100'] = 219
place_with_state_name_gdf.loc['1600000US3150790', 'POP100'] = 74
place_with_state_name_gdf.loc['1600000US3151595', 'POP100'] = 129
place_with_state_name_gdf.loc['1600000US3477270', 'POP100'] = 7299
place_with_state_name_gdf.loc['1600000US3479610', 'POP100'] = 52912
place_with_state_name_gdf.loc['1600000US3626319', 'POP100'] = 2888
place_with_state_name_gdf.loc['1600000US3659531', 'POP100'] = 3160
place_with_state_name_gdf.loc['1600000US3678465', 'POP100'] = 5657
place_with_state_name_gdf.loc['1600000US3683371', 'POP100'] = 377
place_with_state_name_gdf.loc['1600000US3771220', 'POP100'] = 9875
place_with_state_name_gdf.loc['1600000US3771240', 'POP100'] = 392
place_with_state_name_gdf.loc['1600000US3852740', 'POP100'] = 263
place_with_state_name_gdf.loc['1600000US3921434', 'POP100'] = 41302
place_with_state_name_gdf.loc['1600000US3927538', 'POP100'] = 215
place_with_state_name_gdf.loc['1600000US3946872', 'POP100'] = 1405
place_with_state_name_gdf.loc['1600000US3953942', 'POP100'] = 706
place_with_state_name_gdf.loc['1600000US3955776', 'POP100'] = 873
place_with_state_name_gdf.loc['1600000US3958226', 'POP100'] = 223
place_with_state_name_gdf.loc['1600000US3958730', 'POP100'] = 19950
place_with_state_name_gdf.loc['1600000US3981718', 'POP100'] = 14401
place_with_state_name_gdf.loc['1600000US3981732', 'POP100'] = 712
place_with_state_name_gdf.loc['1600000US3986730', 'POP100'] = 8756
place_with_state_name_gdf.loc['1600000US4019850', 'POP100'] = 267
place_with_state_name_gdf.loc['1600000US4038600', 'POP100'] = 711
place_with_state_name_gdf.loc['1600000US4078700', 'POP100'] = 673
place_with_state_name_gdf.loc['1600000US3964346', 'POP100'] = 548
place_with_state_name_gdf.loc['1600000US4210768', 'POP100'] = 5362
place_with_state_name_gdf.loc['1600000US4218736', 'POP100'] = 675
place_with_state_name_gdf.loc['1600000US4222016', 'POP100'] = 1853
place_with_state_name_gdf.loc['1600000US4236816', 'POP100'] = 14044
place_with_state_name_gdf.loc['1600000US4254344', 'POP100'] = 50
place_with_state_name_gdf.loc['1600000US4256432', 'POP100'] = 3349
place_with_state_name_gdf.loc['1600000US4256496', 'POP100'] = 788
place_with_state_name_gdf.loc['1600000US4281328', 'POP100'] = 13176
place_with_state_name_gdf.loc['1600000US4281424', 'POP100'] = 202
place_with_state_name_gdf.loc['1600000US4284272', 'POP100'] = 2638
place_with_state_name_gdf.loc['1600000US4286856', 'POP100'] = 3102
place_with_state_name_gdf.loc['1600000US4816120', 'POP100'] = 3991
place_with_state_name_gdf.loc['1600000US4836092', 'POP100'] = 8154
place_with_state_name_gdf.loc['1600000US4836104', 'POP100'] = 6535
place_with_state_name_gdf.loc['1600000US4848804', 'POP100'] = 74259
place_with_state_name_gdf.loc['1600000US4850760', 'POP100'] = 1314
place_with_state_name_gdf.loc['1600000US4981960', 'POP100'] = 27993
place_with_state_name_gdf.loc['1600000US4982070', 'POP100'] = 9267
place_with_state_name_gdf.loc['1600000US5183248', 'POP100'] = 86
place_with_state_name_gdf.loc['1600000US5548225', 'POP100'] = 2613
place_with_state_name_gdf.loc['1600000US5560200', 'POP100'] = 11179
place_with_state_name_gdf.loc['1600000US5564450', 'POP100'] = 12353


print(
    place_with_state_name_gdf[
        (place_with_state_name_gdf.index.isin(other_state_names_gdf.index))
        #& (place_with_state_name_gdf['POP100'].notnull())
        & (place_with_state_name_gdf['POP100'] >= 5000)
    ].apply(lambda row: '{0:}, {1:} (pop = {2:,.0f})'.format(row['NAME'], row['state_name'], row['POP100']), axis=1).sort_values()#[0:20]
)

states_with_places_named_after_other_states_list = place_with_state_name_gdf[
    (place_with_state_name_gdf.index.isin(other_state_names_gdf.index))
    #& (place_with_state_name_gdf['POP100'].notnull())
    & (place_with_state_name_gdf['POP100'] >= 5000)
]['state_name'].drop_duplicates().tolist()


states_with_both = [x for x in states_with_places_named_after_themselves_list if x in states_with_places_named_after_other_states_list]
states_with_just_themselves = [x for x in states_with_places_named_after_themselves_list if x not in states_with_places_named_after_other_states_list]
states_with_just_other_state_places = [x for x in states_with_places_named_after_other_states_list if x not in states_with_places_named_after_themselves_list]

print('States with both:')
print(states_with_both)
print('\n')
print('States with just places named after themselves:')
print(states_with_just_themselves)
print('\n')
print('States with just places named after other states:')
print(states_with_just_other_state_places)
print('\n')
print('DONE!')

Finding states with place names that include the name of the state...
GEOID
1600000US0609780    California City, California (pop = 14,973)
1600000US0816000    Colorado Springs, Colorado (pop = 478,961)
1600000US1222975          Florida City, Florida (pop = 13,085)
1600000US1639700             Idaho Falls, Idaho (pop = 64,818)
1600000US1938640                Iowa Falls, Iowa (pop = 5,106)
1600000US1938595                Iowa City, Iowa (pop = 74,828)
1600000US2036000           Kansas City, Kansas (pop = 156,607)
1600000US3133705         Nebraska City, Nebraska (pop = 7,222)
1600000US3651000          New York, New York (pop = 8,804,190)
1600000US4055000       Oklahoma City, Oklahoma (pop = 681,054)
1600000US4155200            Oregon City, Oregon (pop = 37,572)
1600000US4872392              Texas City, Texas (pop = 51,898)
1600000US5182000      Virginia Beach, Virginia (pop = 459,470)
1600000US5588200    Wisconsin Rapids, Wisconsin (pop = 18,887)
dtype: object
Finding states with place na

In [ ]:
# print('\n')
# place_with_state_name_gdf[
#     (place_with_state_name_gdf.index.isin(other_state_names_gdf.index))
#     & (place_with_state_name_gdf['POP100'].notnull())
# ].apply(lambda row: '{1:}: {0:} (pop = {2:,.0f})'.format(row['NAME'], row['state_name'], row['POP100']), axis=1).sort_values()



In [ ]:
# s = time.time()
# print('finding bounds...')
# # xlimits = [place_gdf[place_gdf['NAME'] == theplace].bounds['minx'].values[0], place_gdf[place_gdf['NAME'] == theplace].bounds['maxx'].values[0]]
# # ylimits = [place_gdf[place_gdf['NAME'] == theplace].bounds['miny'].values[0], place_gdf[place_gdf['NAME'] == theplace].bounds['maxy'].values[0]]


# # Florence, CO
# xlimits = [-105.14,-105.06]
# ylimits = [38.26,38.41]

# ### Orlando
# # xlimits = [-81.5,-81.3]
# # ylimits = [28.45,28.6]

# aspect_ratio = (xlimits[1] - xlimits[0]) / (ylimits[1] - ylimits[0])


# fig, ax = plt.subplots(1,1,figsize=(24,24/aspect_ratio))

# ax.set_aspect('equal')

# print('plotting major roads...')

# roads_gdf[roads_gdf['RTTYP'] == 'C'].plot(ax=ax, color='black', lw=2)
# roads_gdf[roads_gdf['RTTYP'] == 'S'].plot(ax=ax, color='black', lw=3)
# roads_gdf[roads_gdf['RTTYP'] == 'U'].plot(ax=ax, color='black', lw=4)
# roads_gdf[roads_gdf['RTTYP'] == 'I'].plot(ax=ax, color='black', lw=5)
# roads_gdf[(roads_gdf['FULLNAME'] == 'State Hwy 91')].plot(ax=ax, color='black', lw=5)

# print('plotting minor roads...')
# roads_gdf[(roads_gdf['RTTYP'].isin(['M', 'O']))].plot(ax=ax, color='black', lw=0.25)

# print('plotting unnamed roads...')
# roads_gdf[(roads_gdf['RTTYP'].isnull())].plot(ax=ax, color='black', lw=0.1)



# #state_gdf[state_gdf['STUSPS'] == 'FL'].plot(ax=ax, color='none', edgecolor='black')

# place_gdf.plot(ax=ax, color='none', edgecolor='purple', lw=2)
# place_gdf[place_gdf['NAME'] == theplace].plot(ax=ax, color='none', edgecolor='purple', lw=5)

# print('plotting water areas...')
# water_gdf.plot(ax=ax, color='silver')


# ax.set_xlim(xlimits)
# ax.set_ylim(ylimits)

# # plt.xticks([])
# # plt.yticks([])

# # print('saving...')
# # thisdir = os.getcwd()
# # os.chdir(thisdir)
# #plt.savefig(thisdir+'/city.jpg', format='jpg', dpi=300, bbox_inches='tight')

# e = time.time()
# g += (e-s)
# print('Plotted in {0:.0f} minutes {1:.1f} seconds!'.format(np.floor((e-s)/60), (e-s)%60))


In [ ]:

# & (roads_gdf['FULLNAME'] == 'State Hwy 91')
#roads_gdf.dtypes
#roads_gdf[(roads_gdf['FULLNAME'] == 'State Hwy 91')]
#roads_gdf.groupby('RTTYP').size()

